In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [33]:
df=pd.read_csv("final_train_5.csv")
df=df.drop("Unnamed: 0",1)
df.head()

,SaleDollarCnt,PropertyID,censusblockgroup,Usecode,BedroomCnt,BathroomCnt,FinishedSquareFeet,GarageSquareFeet,LotSizeSquareFeet,StoryCnt,...,ZoneCodeCounty_UR,ZoneCodeCounty_URPSO,ZoneCodeCounty_UV,ZoneCodeCounty_UVEV,ZoneCodeCounty_WD II,Missing ViewType,Missing GarageSquareFeet,Missing BGMedYearBuilt,Missing BGMedRent,Missing BGMedHomeValue
0,285000.0,-0.397302,0.0,0.0,0.633285,-0.375478,-0.320119,-0.049818,-0.197314,-1.012896,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,0.542108,-0.569910,-0.147578,-0.541975,-0.022761
1,309950.0,-0.397299,0.0,0.0,-0.521924,-0.375478,-0.031915,-0.775699,-0.041773,-1.012896,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,-1.844651,-0.569910,-0.147578,-0.541975,-0.022761
2,476000.0,-0.397296,0.0,0.0,0.633285,-1.521526,-0.053264,0.449225,-0.220023,-1.012896,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,0.542108,-0.569910,-0.147578,-0.541975,-0.022761
3,324950.0,-0.397295,0.0,0.0,0.633285,-0.088966,0.384379,-0.514406,-0.005720,-1.012896,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,-1.844651,1.754664,-0.147578,-0.541975,-0.022761
4,325000.0,-0.397294,0.0,0.0,0.633285,-0.661990,-0.512255,-0.357126,-0.170997,0.903396,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,-1.844651,1.754664,-0.147578,-0.541975,-0.022761


In [34]:
X = df.iloc[:,1:]
X.head()

,PropertyID,censusblockgroup,Usecode,BedroomCnt,BathroomCnt,FinishedSquareFeet,GarageSquareFeet,LotSizeSquareFeet,StoryCnt,BuiltYear,...,ZoneCodeCounty_UR,ZoneCodeCounty_URPSO,ZoneCodeCounty_UV,ZoneCodeCounty_UVEV,ZoneCodeCounty_WD II,Missing ViewType,Missing GarageSquareFeet,Missing BGMedYearBuilt,Missing BGMedRent,Missing BGMedHomeValue
0,-0.397302,0.0,0.0,0.633285,-0.375478,-0.320119,-0.049818,-0.197314,-1.012896,-0.371555,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,0.542108,-0.569910,-0.147578,-0.541975,-0.022761
1,-0.397299,0.0,0.0,-0.521924,-0.375478,-0.031915,-0.775699,-0.041773,-1.012896,-0.799322,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,-1.844651,-0.569910,-0.147578,-0.541975,-0.022761
2,-0.397296,0.0,0.0,0.633285,-1.521526,-0.053264,0.449225,-0.220023,-1.012896,-0.728028,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,0.542108,-0.569910,-0.147578,-0.541975,-0.022761
3,-0.397295,0.0,0.0,0.633285,-0.088966,0.384379,-0.514406,-0.005720,-1.012896,-0.478497,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,-1.844651,1.754664,-0.147578,-0.541975,-0.022761
4,-0.397294,0.0,0.0,0.633285,-0.661990,-0.512255,-0.357126,-0.170997,0.903396,-0.977559,...,-0.062438,-0.087976,-0.077958,-0.058854,-0.013139,-1.844651,1.754664,-0.147578,-0.541975,-0.022761


In [35]:
y = pd.DataFrame(df.iloc[:,0])
y.head()

,SaleDollarCnt
0,285000.0
1,309950.0
2,476000.0
3,324950.0
4,325000.0


# LightGBM Final Models:


In [36]:
def cross_validation_function3(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
#         print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain.values.ravel())
        y_pred=model.predict(dfxtest)
        y_true=pd.Series(dfytest.iloc[:,0])
#         pd.Series(y.iloc[:,0])
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
#         print("        FOLD ",f)
#         print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
    return np.mean(scores)
      
        

In [12]:
from lightgbm import LGBMRegressor
import numpy as np
model = LGBMRegressor(num_iterations=900,boosting_type='dart', max_depth=7, learning_rate=0.08, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(num_iterations=900,boosting_type='dart', max_depth=7, learning_rate=0.08, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training Error :  0.09383405657467478


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: U

Final score is:  0.13037873681970596
Testing Cross Validation Error:  0.13037873681970596


In [44]:
def cross_validation_function5(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    mainlist=[]
    for f in range(fold):
#         print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain.values.ravel())
        y_pred=model.predict(dfxtest)
        templist=list(y_pred)
        mainlist=mainlist + templist
        y_true=pd.Series(dfytest.iloc[:,0])
#         pd.Series(y.iloc[:,0])
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
#         print("        FOLD ",f)
#         print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
    return np.mean(scores),mainlist
      
        

In [45]:
# 5    #### 1 star @@@@@
from lightgbm import LGBMRegressor
import numpy as np
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.13, n_estimators=600,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.13, n_estimators=600,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore,newprediction1=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.0910829674581461
Final score is:  0.13003222743241366
Testing Cross Validation Error:  0.13003222743241366


In [48]:
maindf=pd.DataFrame()


In [51]:
maindf["Model 1"]=newprediction1
maindf.head()

,Model 1
0,262521.878520
1,332928.985356
2,490602.175458
3,342448.554500
4,305932.030521


In [52]:
# 12   ^^^^^  2 star   ^^^^^^
from lightgbm import LGBMRegressor
import numpy as np
model = LGBMRegressor(boosting_type='dart', max_depth=9, learning_rate=0.19, n_estimators=400,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=9, learning_rate=0.19, n_estimators=400,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore,newprediction2=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.09173034223345002
Final score is:  0.1294838650847539
Testing Cross Validation Error:  0.1294838650847539


In [53]:
# 11     ########## 3 star ##########
from lightgbm import LGBMRegressor
import numpy as np
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.24, n_estimators=350,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.24, n_estimators=350,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore,newprediction3=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.08780665488601577
Final score is:  0.1301254008919178
Testing Cross Validation Error:  0.1301254008919178


In [54]:
# 4              ###### 4 star $$$$$$
model = LGBMRegressor(boosting_type='dart',num_iterations=900 ,max_depth=7, learning_rate=0.09, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart',num_iterations=900 ,max_depth=7, learning_rate=0.09, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore,newprediction4=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training Error :  0.09118468888695819


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: U

Final score is:  0.1301376781265132
Testing Cross Validation Error:  0.1301376781265132


In [55]:
# 25      %%% 5  star %%%%

model = LGBMRegressor(boosting_type='dart', num_leaves=29,min_data_in_leaf=15 ,max_depth=7, learning_rate=0.09, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', num_leaves=29,min_data_in_leaf=15 ,max_depth=7, learning_rate=0.09, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore,newprediction5=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)



        

Training Error :  0.0906421739064629
Final score is:  0.12912559913782506
Testing Cross Validation Error:  0.12912559913782506


In [56]:

# 22     ######6 star$$$$$$$$$

model = LGBMRegressor(boosting_type='dart', num_leaves=25, max_depth=7, learning_rate=0.13, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', num_leaves=25, max_depth=7, learning_rate=0.13, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore,newprediction6=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)



Training Error :  0.08639806154333846
Final score is:  0.12946930176990257
Testing Cross Validation Error:  0.12946930176990257


In [57]:
# 16--Final Gradient Booster ALone   ##### 7 star #####

from sklearn import ensemble
from sklearn.model_selection import cross_val_score  
model = ensemble.GradientBoostingRegressor(n_estimators= 300, max_depth=7,learning_rate=0.054 , loss= 'ls',random_state=30)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = ensemble.GradientBoostingRegressor(n_estimators= 300, max_depth=7,learning_rate=0.054 , loss= 'ls',random_state=30)
newscore,newprediction7=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


              

    

Training Error :  0.07584326247482823
Final score is:  0.133919734864522
Testing Cross Validation Error:  0.133919734864522


In [58]:
# Zillow 1.4$   ##### 8 star #####
model = XGBRegressor(max_depth=8,learning_rate=0.04, n_estimators=500,booster='gbtree',random_state=30)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = XGBRegressor(max_depth=8,learning_rate=0.04, n_estimators=500,booster='gbtree',random_state=30)
newscore,newprediction8=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)




Training Error :  0.06381928266400659
Final score is:  0.13411487443671416
Testing Cross Validation Error:  0.13411487443671416


In [60]:

# Zillow 1.4$##   ##### 9 star #####
model = XGBRegressor(max_depth=8,learning_rate=0.081, n_estimators=100,booster='gbtree',random_state=30)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = XGBRegressor(max_depth=8,learning_rate=0.081, n_estimators=100,booster='gbtree',random_state=30)
newscore,newprediction9=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)



Training Error :  0.07988238944609445
Final score is:  0.13469952012166053
Testing Cross Validation Error:  0.13469952012166053


In [61]:
# Zillow -----> ##### 10 star #####
model = RandomForestRegressor(max_depth=18, random_state=30,n_estimators=91)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = RandomForestRegressor(max_depth=18, random_state=30,n_estimators=91)
newscore,newprediction10=cross_validation_function5(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)
            


Training Error :  0.05291163508976431
Final score is:  0.14433978908042905
Testing Cross Validation Error:  0.14433978908042905


In [62]:
maindf["Model 1"]=newprediction1
maindf["Model 2"]=newprediction2
maindf["Model 3"]=newprediction3
maindf["Model 4"]=newprediction4
maindf["Model 5"]=newprediction5
maindf["Model 6"]=newprediction6
maindf["Model 7"]=newprediction7
maindf["Model 8"]=newprediction8
maindf["Model 9"]=newprediction9
maindf["Model 10"]=newprediction10

maindf.head()

,Model 1,Model 2,Model 3,Model 4,Model 5,Model 6,Model 7,Model 8,Model 9,Model 10
0,262521.878520,255326.401497,260132.800960,262287.383701,260219.968623,263179.585892,263121.673757,258759.75000,257875.09375,281089.981962
1,332928.985356,315633.720081,322205.082089,325354.683470,322101.929533,309633.844363,340416.785533,319191.68750,336675.40625,352975.002584
2,490602.175458,481046.483351,487062.807700,474635.051386,488908.086621,498633.683838,535588.582278,515659.00000,518264.93750,563381.119324
3,342448.554500,352571.162562,333338.046314,353287.601711,342887.897821,346680.052506,330273.702360,323240.62500,345656.93750,378295.022702
4,305932.030521,289842.010467,314063.619298,311320.346878,306490.238979,308895.932337,320440.796837,312079.84375,315151.65625,351184.629669


In [89]:
maindf.insert(loc=0, column='y', value=y)


In [90]:
maindf.head(10)

,y,Model 1,Model 2,Model 3,Model 4,Model 5,Model 6,Model 7,Model 8,Model 9,Model 10
0,285000.0,262521.878520,255326.401497,260132.800960,262287.383701,260219.968623,263179.585892,263121.673757,258759.75000,257875.09375,281089.981962
1,309950.0,332928.985356,315633.720081,322205.082089,325354.683470,322101.929533,309633.844363,340416.785533,319191.68750,336675.40625,352975.002584
2,476000.0,490602.175458,481046.483351,487062.807700,474635.051386,488908.086621,498633.683838,535588.582278,515659.00000,518264.93750,563381.119324
3,324950.0,342448.554500,352571.162562,333338.046314,353287.601711,342887.897821,346680.052506,330273.702360,323240.62500,345656.93750,378295.022702
4,325000.0,305932.030521,289842.010467,314063.619298,311320.346878,306490.238979,308895.932337,320440.796837,312079.84375,315151.65625,351184.629669
5,235000.0,272752.681107,269312.615583,276245.691434,280523.589189,273453.651624,270881.107695,279287.818052,289651.15625,292718.31250,299094.880491
6,653295.0,585881.963233,553185.387005,563095.660713,579316.132379,574940.572297,571162.390259,622298.972842,618828.43750,641031.18750,652416.316586
7,370000.0,397330.304246,372078.594993,343953.062185,367264.176228,363194.117264,346464.228644,420785.757512,347469.50000,354420.46875,332381.884085
8,334950.0,259620.651275,268349.600280,258949.233070,267060.588909,265065.518003,260267.238947,245933.313170,227614.96875,275968.71875,302956.600612
9,625000.0,579692.710000,583867.565807,587678.424641,580253.970998,573871.958624,567366.245256,618893.239209,617565.56250,624878.75000,659900.964584


In [66]:
maindf.corr()

,y,Model 1,Model 2,Model 3,Model 4,Model 5,Model 6,Model 7,Model 8,Model 9,Model 10
y,1.000000,0.902615,0.902313,0.903493,0.902737,0.901657,0.903730,0.892681,0.888974,0.891214,0.893118
Model 1,0.902615,1.000000,0.997024,0.996247,0.997808,0.996032,0.996784,0.980612,0.971332,0.973502,0.978185
Model 2,0.902313,0.997024,1.000000,0.996003,0.996968,0.995125,0.996271,0.980882,0.972258,0.974200,0.978884
Model 3,0.903493,0.996247,0.996003,1.000000,0.996353,0.994586,0.996167,0.979530,0.971437,0.973341,0.977717
Model 4,0.902737,0.997808,0.996968,0.996353,1.000000,0.996068,0.996968,0.980352,0.971513,0.973611,0.978701
Model 5,0.901657,0.996032,0.995125,0.994586,0.996068,1.000000,0.995711,0.980078,0.971552,0.974032,0.977760
Model 6,0.903730,0.996784,0.996271,0.996167,0.996968,0.995711,1.000000,0.979179,0.970682,0.972447,0.975243
Model 7,0.892681,0.980612,0.980882,0.979530,0.980352,0.980078,0.979179,1.000000,0.977881,0.979059,0.985093
Model 8,0.888974,0.971332,0.972258,0.971437,0.971513,0.971552,0.970682,0.977881,1.000000,0.995023,0.975779
Model 9,0.891214,0.973502,0.974200,0.973341,0.973611,0.974032,0.972447,0.979059,0.995023,1.000000,0.978260


In [88]:
secdf=pd.DataFrame()
secdf["Model 1 Diff"]=maindf["Model 1"] - maindf['y']
secdf["Model 2 Diff"]=maindf["Model 2"] - maindf['y']
secdf["Model 3 Diff"]=maindf["Model 3"] - maindf['y']
secdf["Model 4 Diff"]=maindf["Model 4"] - maindf['y']
secdf["Model 5 Diff"]=maindf["Model 5"] - maindf['y']
secdf["Model 6 Diff"]=maindf["Model 6"] - maindf['y']
secdf["Model 7 Diff"]=maindf["Model 7"] - maindf['y']
secdf["Model 8 Diff"]=maindf["Model 8"] - maindf['y']
secdf["Model 9 Diff"]=maindf["Model 9"] - maindf['y']
secdf["Model 10 Diff"]=maindf["Model 10"] - maindf['y']

secdf.head(10)

,Model 1 Diff,Model 2 Diff,Model 3 Diff,Model 4 Diff,Model 5 Diff,Model 6 Diff,Model 7 Diff,Model 8 Diff,Model 9 Diff,Model 10 Diff
0,-22478.121480,-29673.598503,-24867.199040,-22712.616299,-24780.031377,-21820.414108,-21878.326243,-26240.25000,-27124.90625,-3910.018038
1,22978.985356,5683.720081,12255.082089,15404.683470,12151.929533,-316.155637,30466.785533,9241.68750,26725.40625,43025.002584
2,14602.175458,5046.483351,11062.807700,-1364.948614,12908.086621,22633.683838,59588.582278,39659.00000,42264.93750,87381.119324
3,17498.554500,27621.162562,8388.046314,28337.601711,17937.897821,21730.052506,5323.702360,-1709.37500,20706.93750,53345.022702
4,-19067.969479,-35157.989533,-10936.380702,-13679.653122,-18509.761021,-16104.067663,-4559.203163,-12920.15625,-9848.34375,26184.629669
5,37752.681107,34312.615583,41245.691434,45523.589189,38453.651624,35881.107695,44287.818052,54651.15625,57718.31250,64094.880491
6,-67413.036767,-100109.612995,-90199.339287,-73978.867621,-78354.427703,-82132.609741,-30996.027158,-34466.56250,-12263.81250,-878.683414
7,27330.304246,2078.594993,-26046.937815,-2735.823772,-6805.882736,-23535.771356,50785.757512,-22530.50000,-15579.53125,-37618.115915
8,-75329.348725,-66600.399720,-76000.766930,-67889.411091,-69884.481997,-74682.761053,-89016.686830,-107335.03125,-58981.28125,-31993.399388
9,-45307.290000,-41132.434193,-37321.575359,-44746.029002,-51128.041376,-57633.754744,-6106.760791,-7434.43750,-121.25000,34900.964584


In [112]:
sec=pd.DataFrame()
sec["Model 1 Diff"]=np.abs(maindf["Model 1"] - maindf['y'])/maindf['y']
sec["Model 2 Diff"]=np.abs(maindf["Model 2"] - maindf['y'])/maindf['y']
sec["Model 3 Diff"]=np.abs(maindf["Model 3"] - maindf['y'])/maindf['y']
sec["Model 4 Diff"]=np.abs(maindf["Model 4"] - maindf['y'])/maindf['y']
sec["Model 5 Diff"]=np.abs(maindf["Model 5"] - maindf['y'])/maindf['y']
sec["Model 6 Diff"]=np.abs(maindf["Model 6"] - maindf['y'])/maindf['y']
sec["Model 7 Diff"]=np.abs(maindf["Model 7"] - maindf['y'])/maindf['y']
sec["Model 8 Diff"]=np.abs(maindf["Model 8"] - maindf['y'])/maindf['y']
sec["Model 9 Diff"]=np.abs(maindf["Model 9"] - maindf['y'])/maindf['y']
sec["Model 10 Diff"]=np.abs(maindf["Model 10"] - maindf['y'])/maindf['y']

sec

,Model 1 Diff,Model 2 Diff,Model 3 Diff,Model 4 Diff,Model 5 Diff,Model 6 Diff,Model 7 Diff,Model 8 Diff,Model 9 Diff,Model 10 Diff
0,0.078871,0.104118,0.087253,0.079693,0.086947,0.076563,0.076766,0.092071,0.095175,0.013719
1,0.074138,0.018338,0.039539,0.049701,0.039206,0.001020,0.098296,0.029817,0.086225,0.138813
2,0.030677,0.010602,0.023241,0.002868,0.027118,0.047550,0.125186,0.083317,0.088792,0.183574
3,0.053850,0.085001,0.025813,0.087206,0.055202,0.066872,0.016383,0.005260,0.063723,0.164164
4,0.058671,0.108178,0.033650,0.042091,0.056953,0.049551,0.014028,0.039754,0.030303,0.080568
5,0.160650,0.146011,0.175514,0.193717,0.163633,0.152686,0.188459,0.232558,0.245610,0.272744
6,0.103189,0.153238,0.138068,0.113240,0.119937,0.125721,0.047446,0.052758,0.018772,0.001345
7,0.073866,0.005618,0.070397,0.007394,0.018394,0.063610,0.137259,0.060893,0.042107,0.101671
8,0.224897,0.198837,0.226902,0.202685,0.208642,0.222967,0.265761,0.320451,0.176090,0.095517
9,0.072492,0.065812,0.059715,0.071594,0.081805,0.092214,0.009771,0.011895,0.000194,0.055842


In [91]:
secdf.describe()

,Model 1 Diff,Model 2 Diff,Model 3 Diff,Model 4 Diff,Model 5 Diff,Model 6 Diff,Model 7 Diff,Model 8 Diff,Model 9 Diff,Model 10 Diff
count,1.158500e+04,1.158500e+04,1.158500e+04,1.158500e+04,1.158500e+04,1.158500e+04,1.158500e+04,1.158500e+04,1.158500e+04,1.158500e+04
mean,-1.071248e+04,-1.651190e+04,-1.003327e+04,-9.459277e+03,-1.264379e+04,-1.145299e+04,-3.188869e+03,-5.163294e+03,-5.586760e+03,5.904763e+03
std,1.970692e+05,1.974009e+05,1.962238e+05,1.969501e+05,1.979846e+05,1.959908e+05,2.066466e+05,2.099215e+05,2.077299e+05,2.059482e+05
min,-3.896860e+06,-4.081291e+06,-3.866715e+06,-3.827125e+06,-3.925216e+06,-4.015964e+06,-4.971180e+06,-4.790067e+06,-4.790068e+06,-4.500315e+06
25%,-4.723497e+04,-5.164920e+04,-4.681198e+04,-4.630585e+04,-4.829316e+04,-4.767781e+04,-4.083493e+04,-4.173834e+04,-4.082184e+04,-3.374074e+04
50%,-2.554371e+03,-6.355793e+03,-2.939206e+03,-1.364099e+03,-3.452520e+03,-3.034910e+03,1.904575e+03,2.072094e+03,2.730344e+03,9.930012e+03
75%,4.084156e+04,3.663483e+04,4.189445e+04,4.201571e+04,3.891881e+04,4.004456e+04,4.879335e+04,4.881912e+04,4.929825e+04,6.325047e+04
max,2.665931e+06,2.558717e+06,2.613563e+06,2.739440e+06,2.599624e+06,2.709411e+06,2.693534e+06,3.015724e+06,3.182957e+06,2.231159e+06


In [105]:
import numpy as np, numpy.random
w1=np.random.dirichlet(np.ones(10),size=1)
w1=list(w1[0])
w1

[0.007594255150633928,
 0.034407444944772395,
 0.22646548409025372,
 0.022501028044347643,
 0.09559911771351595,
 0.034889144045619026,
 0.381061682404559,
 0.004495723042812846,
 0.10096806187135418,
 0.09201805869213116]

In [14]:
# 6
from lightgbm import LGBMRegressor
import numpy as np
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.13, n_estimators=500,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.13, n_estimators=500,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.09440770408188659
Final score is:  0.1299319033440164
Testing Cross Validation Error:  0.1299319033440164


In [15]:
# 7
from lightgbm import LGBMRegressor
import numpy as np
model = LGBMRegressor(boosting_type='dart', max_depth=7, learning_rate=0.14, n_estimators=400,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=7, learning_rate=0.14, n_estimators=400,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.09756882300621789
Final score is:  0.1301227287454838
Testing Cross Validation Error:  0.1301227287454838


In [16]:
# 12   ^^^^^  2   ^^^^^^
from lightgbm import LGBMRegressor
import numpy as np
model = LGBMRegressor(boosting_type='dart', max_depth=9, learning_rate=0.19, n_estimators=400,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=9, learning_rate=0.19, n_estimators=400,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.09173034223345002
Final score is:  0.1294838650847539
Testing Cross Validation Error:  0.1294838650847539


In [17]:
# 11     ##########3##########
from lightgbm import LGBMRegressor
import numpy as np
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.24, n_estimators=350,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.24, n_estimators=350,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.08780665488601577
Final score is:  0.1301254008919178
Testing Cross Validation Error:  0.1301254008919178


In [6]:
from lightgbm import LGBMRegressor
import numpy as np
model = LGBMRegressor(num_iterations=900,boosting_type='dart', max_depth=7, learning_rate=0.09, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(num_iterations=900,boosting_type='dart', max_depth=7, learning_rate=0.08, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training Error :  0.09118468888695819


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: U

Final score is:  0.13037873681970596
Testing Cross Validation Error:  0.13037873681970596


In [7]:

model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.13, n_estimators=600,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.13, n_estimators=600,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.0910829674581461
Final score is:  0.13003222743241366
Testing Cross Validation Error:  0.13003222743241366


In [8]:
# 12
model = LGBMRegressor(boosting_type='dart', max_depth=9, learning_rate=0.19, n_estimators=400,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=9, learning_rate=0.19, n_estimators=400,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.09173034223345002
Final score is:  0.1294838650847539
Testing Cross Validation Error:  0.1294838650847539


In [9]:
# 6
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.13, n_estimators=500,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', max_depth=8, learning_rate=0.13, n_estimators=500,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.09440770408188659
Final score is:  0.1299319033440164
Testing Cross Validation Error:  0.1299319033440164


In [10]:
# 4              ######4$$$$$$
model = LGBMRegressor(boosting_type='dart',num_iterations=900 ,max_depth=7, learning_rate=0.09, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart',num_iterations=900 ,max_depth=7, learning_rate=0.09, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training Error :  0.09118468888695819


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:116: U

Final score is:  0.1301376781265132
Testing Cross Validation Error:  0.1301376781265132


In [11]:
# 25      %%%5%%%%

model = LGBMRegressor(boosting_type='dart', num_leaves=29,min_data_in_leaf=15 ,max_depth=7, learning_rate=0.09, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', num_leaves=29,min_data_in_leaf=15 ,max_depth=7, learning_rate=0.09, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.0906421739064629
Final score is:  0.12912559913782506
Testing Cross Validation Error:  0.12912559913782506


In [13]:
# 22     ######6$$$$$$$$$

model = LGBMRegressor(boosting_type='dart', num_leaves=25, max_depth=7, learning_rate=0.13, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = LGBMRegressor(boosting_type='dart', num_leaves=25, max_depth=7, learning_rate=0.13, n_estimators=1000,objective='regression',metric='mape',tree_learner='serial',random_state=30, n_jobs=-1 )
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.08639806154333846
Final score is:  0.12946930176990257
Testing Cross Validation Error:  0.12946930176990257


# Gradient Booster Final 

In [15]:
# 16--Final Gradient Booster ALone   #####7#####

from sklearn import ensemble
from sklearn.model_selection import cross_val_score  
model = ensemble.GradientBoostingRegressor(n_estimators= 300, max_depth=7,learning_rate=0.054 , loss= 'ls',random_state=30)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = ensemble.GradientBoostingRegressor(n_estimators= 300, max_depth=7,learning_rate=0.054 , loss= 'ls',random_state=30)
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


              

Training Error :  0.07584326247482823
Final score is:  0.133919734864522
Testing Cross Validation Error:  0.133919734864522


# Final XGBoost MODEL :

In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost import XGBRegressor
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [21]:
# Zillow 1.2
model = XGBRegressor(max_depth=9,learning_rate=0.04, n_estimators=178,booster='gbtree',random_state=30)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = XGBRegressor(max_depth=9,learning_rate=0.04, n_estimators=178,booster='gbtree',random_state=30)
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)


            

Training Error :  0.07091322221615406
Final score is:  0.13619988841724287
Testing Cross Validation Error:  0.13619988841724287


In [22]:
# Zillow 1.3
model = XGBRegressor(max_depth=8,learning_rate=0.1, n_estimators=200,booster='gbtree',random_state=30)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = XGBRegressor(max_depth=8,learning_rate=0.1, n_estimators=200,booster='gbtree',random_state=30)
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)

Training Error :  0.06262514875825706
Final score is:  0.13649872671587376
Testing Cross Validation Error:  0.13649872671587376


In [23]:
# Zillow 1.4
model = XGBRegressor(max_depth=8,learning_rate=0.04, n_estimators=100,booster='gbtree',random_state=30)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = XGBRegressor(max_depth=8,learning_rate=0.04, n_estimators=100,booster='gbtree',random_state=30)
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)

Training Error :  0.09372542563129925
Final score is:  0.13494654749486582
Testing Cross Validation Error:  0.13494654749486582


In [25]:
# Zillow 1.4$   #####8#####
model = XGBRegressor(max_depth=8,learning_rate=0.04, n_estimators=500,booster='gbtree',random_state=30)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = XGBRegressor(max_depth=8,learning_rate=0.04, n_estimators=500,booster='gbtree',random_state=30)
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)

Training Error :  0.06381928266400659
Final score is:  0.13411487443671416
Testing Cross Validation Error:  0.13411487443671416


In [26]:
# Zillow 1.4$##   #####9#####
model = XGBRegressor(max_depth=8,learning_rate=0.081, n_estimators=100,booster='gbtree',random_state=30)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = XGBRegressor(max_depth=8,learning_rate=0.081, n_estimators=100,booster='gbtree',random_state=30)
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)

Training Error :  0.07988238944609445
Final score is:  0.13469952012166053
Testing Cross Validation Error:  0.13469952012166053


# Random Forest FINAL

In [27]:
# Zillow -----> #####10#####
model = RandomForestRegressor(max_depth=18, random_state=30,n_estimators=91)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = RandomForestRegressor(max_depth=18, random_state=30,n_estimators=91)
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)
            

Training Error :  0.05291163508976431
Final score is:  0.14433978908042905
Testing Cross Validation Error:  0.14433978908042905


In [30]:
# Zillow -----> 0.1725
model = RandomForestRegressor(max_depth=9, random_state=30,n_estimators=500)
model.fit(X,y.values.ravel())
y_pred=model.predict(X)
y_true=pd.Series(y.iloc[:,0])
print("Training Error : ",np.mean(np.abs((y_true - y_pred) / y_true)))
model = RandomForestRegressor(max_depth=6, random_state=30,n_estimators=78)
newscore=cross_validation_function3(X,y,model,5)
print("Testing Cross Validation Error: ",newscore)
            

Training Error :  0.1184269569098208
Final score is:  0.1725050602189087
Testing Cross Validation Error:  0.1725050602189087


In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor


In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(X,y, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)


In [ ]:
# wights_file = 'Weights-00500--141345.79756.hdf5' # choose the best checkpoint 
# NN_model.load_weights(wights_file) # load it
# NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
y_pred=NN_model.predict(X)


In [ ]:
temp=pd.DataFrame(y_pred)
temp[2]=y_true
temp[3]=temp[0]-temp[2]
temp

In [ ]:
y_true=y
print(np.mean(np.abs((y_true - y_pred) / y_true)))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import numpy as np
knn = KNeighborsRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
y_true=y_test
print(np.mean(np.abs((y_true - y_pred) / y_true)) )
output=pd.DataFrame(data=y_true)
output["KNN_pred"]=y_pred
output.head()

In [ ]:
output.shape

In [ ]:
def cross_validation_function(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
        print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain)
        y_pred=model.predict(dfxtest)
        y_true=dfytest
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
        print("        FOLD ",f)
        print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
      
        

In [ ]:
from sklearn.model_selection import cross_val_score
for n in range(1,30):
    knn = KNeighborsRegressor(n_neighbors=n,weights="distance")
    print("N=",n)
    cross_validation_function(X,y,knn,10)


In [ ]:
from sklearn.model_selection import cross_val_score
for n in range(1,30):
    knn = KNeighborsRegressor(n_neighbors=n,weights="distance")
    print("N=",n)
    cross_validation_function(X,y,knn,5)


# Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

In [ ]:
def cross_validation_function2(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
#         print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain)
        y_pred=model.predict(dfxtest)
        y_true=pd.Series(dfytest.iloc[:,0])
#         pd.Series(y.iloc[:,0])
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
        print("        FOLD ",f)
        print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
    return np.mean(scores)
      
        

In [ ]:
from sklearn.model_selection import cross_val_score
min=1
sc=[]
for d in range(1,30):
    tree = DecisionTreeRegressor(max_depth=d,criterion='mae')
    print("Depth =",d)
    newscore=cross_validation_function2(X,y,tree,5)
    sc.append(newscore)
    if(newscore<min):
        min=newscore
        print("Min CHANGED TO : ",min," at d =",d)
print(min(sc))


In [ ]:
from sklearn.model_selection import cross_val_score
sc=[]
min=1
for d in range(2,30):
    for nest in range(10,500,10):
        forest = RandomForestRegressor(max_depth=d, random_state=30,n_estimators=nest)
        print("N Estimators is : ",nest)
        print("Depth =",d)
        newscore=cross_validation_function3(X,y,forest,5)
        sc.append(newscore)
        if(newscore<min):
            min=newscore
            print("Min CHANGED TO : ",min," at d =",d," and n est = ",nest)
print(min(sc))
    # print(regr.feature_importances_)

Random Forest Regressor

In [ ]:
from sklearn.model_selection import cross_val_score
for d in range(1,30):
    for nest in range(10,200,10):
    forest = RandomForestRegressor(max_depth=d, random_state=30,n_estimators=100)
    print("Depth =",d)
    cross_validation_function(X,y,forest,5)
# print(regr.feature_importances_)

# Gradient Boosting Regressor

In [ ]:
def cross_validation_function3(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
#         print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain.values.ravel())
        y_pred=model.predict(dfxtest)
        y_true=pd.Series(dfytest.iloc[:,0])
#         pd.Series(y.iloc[:,0])
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
        print("        FOLD ",f)
        print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
    return np.mean(scores)
      
        

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
sc=[]
min=1
for nest in range(10,500,10):
    for d in range(5,25):
        for l in np.arange(0.01,0.5,0.005):
            clf = ensemble.GradientBoostingRegressor(n_estimators= nest, max_depth=d,learning_rate=l , loss= 'ls',random_state=30)
            print("N Estimators is : ",nest)
            print("Depth =",d)
            newscore=cross_validation_function3(X,y,clf,5)
            sc.append(newscore)
            if(newscore<min):
                min=newscore
                print("Min CHANGED TO : ",min," at d =",d," and n est = ",nest,"l =",l)
# print(min(sc))


 


In [ ]:
def cross_validation_function3(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
#         print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain.values.ravel())
        y_pred=model.predict(dfxtest)
        y_true=pd.Series(dfytest.iloc[:,0])
#         pd.Series(y.iloc[:,0])
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
#         print("        FOLD ",f)
#         print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
    return np.mean(scores)
      
        

In [ ]:
from xgboost import XGBRegressor
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
sc=[]
min=1
booster=['gbtree','gblinear','dart']

for nest in range(1,100,1):
    for b in booster:
        for l in np.arange(0.01,0.8,0.005):
            for d in range(3,30):
                xgbreg = XGBRegressor(max_depth=d,learning_rate=l, n_estimators=nest,booster=b,random_state=30)
                print("N Estimators is : ",nest)
                print("Depth =",d)
                print("Learning Rate: ",l)
                print("Booster : ",b)
                newscore=cross_validation_function3(X,y,xgbreg,5)
                sc.append(newscore)
                if(newscore<min):
                    min=newscore
                    print("Min CHANGED TO : ",min," at d =",d," and n est = ",nest,"l =",l," b = ",b)


In [ ]:
max_depth = m,

In [ ]:
for d in range(1,10):
    

In [ ]:
from xgboost import XGBRegressor
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
sc=[]
min=1
booster=['gbtree','dart']
for d in range(7,10):
    for nest in range(19,50,1):
        for b in booster:
            for l in np.arange(0.01,0.9,0.01):
                xgbreg = XGBRegressor(max_depth=d,learning_rate=l, n_estimators=nest,booster=b,random_state=30)
                print("N Estimators is : ",nest)
                print("Depth =",d)
                print("Learning Rate: ",l)
                print("Booster : ",b)
                newscore=cross_validation_function3(X,y,xgbreg,5)
                sc.append(newscore)
                if(newscore<min):
                    min=newscore
                    print("Min CHANGED TO : ",min," at d =",d," and n est = ",nest,"l =",l," b = ",b)


In [ ]:
def cross_validation_function3(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
#         print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain.values.ravel())
        y_pred=model.predict(dfxtest)
        y_true=pd.Series(dfytest.iloc[:,0])
#         pd.Series(y.iloc[:,0])
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
#         print("        FOLD ",f)
#         print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
    return np.mean(scores)
      
        

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
sc=[]
min=1

for nest in range(100,1000,70):
    for d in range(3,25):
        for l in np.arange(0.1,0.9,0.1):
            clf = ensemble.GradientBoostingRegressor(n_estimators= nest, max_depth=d,learning_rate=l , loss= 'ls',random_state=30)
            print("N Estimators is : ",nest)
            print("Depth =",d)
            newscore=cross_validation_function3(X,y,clf,5)
            sc.append(newscore)
            if(newscore<min):
                min=newscore
                print("Min CHANGED TO : ",min," at d =",d," and n est = ",nest,"l =",l)



In [ ]:

# This is test for Zillow 1.1 d=7 nest=??(200)  and l= 0.04
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
sc=[]
min=1
# for d in range(1,30):
for n in range(100,1000,50):
#     for l in np.arange(0.01,0.9,0.005):
    clf = ensemble.GradientBoostingRegressor(n_estimators= n, max_depth=7,learning_rate=0.04 , loss= 'ls',random_state=30)
    print("N Estimators is : ",n)
#     print("learning rate  =",l)
    newscore=cross_validation_function3(X,y,clf,5)
    sc.append(newscore)
    if(newscore<min):
        min=newscore
        print("Min CHANGED TO : ",min," and n est = ",n)



 
#   you need to loop for l --> learning rate too 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
sc=[]
min=1
for d in range(2,30):
    for nest in range(100,1000,50):
        forest = RandomForestRegressor(max_depth=d, random_state=30,n_estimators=nest)
        print("N Estimators is : ",nest)
        print("Depth =",d)
        newscore=cross_validation_function3(X,y,forest,5)
        sc.append(newscore)
        if(newscore<min):
            min=newscore
            print("Min CHANGED TO : ",min," at d =",d," and n est = ",nest)
print(min(sc))
    # print(regr.feature_importances_)


In [ ]:
from sklearn.svm import SVR
Cs = [0.001, 0.01, 0.1, 1, 10]
# gammas = [0.001, 0.01, 0.1, 1]
gammas = [0.1, 1]

ds =[3,4,5,6,7]
sc=[]
min=1
for d in ds:
    for g in gammas:
        for c in Cs:
            model = SVR(kernel='rbf', degree=d, gamma=g, coef0=0.0, tol=0.001, C=c, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
        #     xgbreg = XGBRegressor(max_depth=d,learning_rate=l, n_estimators=nest,booster='gbtree',random_state=30)
        #     print("N Estimators is : ",nest)
            print("Degree =",d)
            print("Gamma: ",g)
            print("C : ",c)
            newscore=cross_validation_function3(X,y,model,5)
            sc.append(newscore)
            if(newscore<min):
                min=newscore
                print("Min CHANGED TO : ",min," at Degree =",d,"Gamma = ",g,"C = ",c)

In [ ]:
def cross_validation_function3(X,y,model,fold):
    size=int(len(X)/fold)
    l=0
    u=size
    scores=[]
    for f in range(fold):
#         print("l =",l,"u =",u,"")
        dfxtest=X.iloc[l:u,:]
        dfytest=y.iloc[l:u,:]
        dfxtrain=X.drop(X.iloc[l:u].index,axis=0)
        dfytrain=y.drop(X.iloc[l:u].index,axis=0)
        model.fit(dfxtrain,dfytrain.values.ravel())
        y_pred=model.predict(dfxtest)
        y_true=pd.Series(dfytest.iloc[:,0])
#         pd.Series(y.iloc[:,0])
#         print(y_true)
#         print(y_pred)
        scores.append(np.mean(np.abs((y_true - y_pred) / y_true)))
#         print("        FOLD ",f)
#         print("        --> Score :", (np.mean(np.abs((y_true - y_pred) / y_true))))
        l=l+size
        u=u+size
    print("Final score is: ",np.mean(scores))
    return np.mean(scores)
      
        

In [ ]:
from lightgbm import LGBMRegressor
import numpy as np
boosting_type_set1=['gbdt','gbrt','dart']
boosting_type_set2=['gbdt']
m=20
n=31
min_data_in_leafset=[15,20,25]
tree_learner_set =['serial','feature','data','voting']
sc=[]
min = 1
for nest in range(900,100,-200):
    for d in range(7,10):
        for l in np.arange(0.005,0.22,0.002):
            for iter in range(200,10000,500):
                for b in boosting_type_set2:
                    for m in min_data_in_leafset:
                        for n in [21,31,41]:
#                 for n in range(25,48,4):
#                     for m in range(5,30,5):
#                 for t in tree_learner_set:
                            lgbm = LGBMRegressor(num_iterations=iter,boosting_type=b, num_leaves=n, max_depth=d, learning_rate=l, n_estimators=nest,objective='regression',metric='mape',tree_learner='serial',min_data_in_leaf=m,random_state=30, n_jobs=-1 )
                            print("N Estimators is : ",nest)
                            print("Depth =",d)
                            print("Learning Rate: ",l)
#                             print("b = ",b)
                            print("NO of iterations :",iter)
                            print("no of leaves is :",n)
                            print("minimum data in leaf =",m)
            #                     print("tree learner set is : ",t)

                            print("Booster : ",b)
                            newscore=cross_validation_function3(X,y,lgbm,5)
                            print("\n\n")
                            sc.append(newscore)
                            if(newscore<min):
                                min=newscore
                #                 print("Min CHANGED TO : ",min,"AND  n est = ",nest,"l = ",l)
                                print("Min CHANGED TO : ",min," at d =",d," and n est = ",nest,"l =",l,"b = ",b,"no of iterations = ",iter,"n = ",n,"m = ",m)
                                print("\n\n")


In [113]:
w=[1/11]*11
w

[0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091]